tobac example: Tracking of precipitation features
===
This example notebook demonstrates the use of tobac to track precipitation features from isolated deep convective clouds.

The simulation results used in this example were performed as part of the ACPC deep convection intercomparison  case study (http://acpcinitiative.org/Docs/ACPC_DCC_Roadmap_171019.pdf) with WRF using the Morrison microphysics scheme.

The data used in this example is downloaded from "zenodo link" automatically as part of the notebooks (This only has to be done once for all the tobac example notebooks).

**Import necessary python libraries:**

In [1]:
!pip install --upgrade ../../../../tobac/

Processing /home/heikenfeld-l/Uni_Oxford/cp/Projects/Software/tobac/tobac
  Created wheel for tobac: filename=tobac-0.70.dev152+g278252a.d20200713-py3-none-any.whl size=43748 sha256=c498ab6089407f82051c36a5ae0f7709e0e0ca3a8e7c45be896dc6ad266b9fa0
  Stored in directory: /tmp/pip-ephem-wheel-cache-wvpo8cb_/wheels/59/4f/06/5306caddafbee06b22a3134ad95ac0bf0cef03def636c34cd7
Successfully built tobac
  Attempting uninstall: tobac
    Found existing installation: tobac 0.70.dev152+g278252a.d20200713
    Uninstalling tobac-0.70.dev152+g278252a.d20200713:
      Successfully uninstalled tobac-0.70.dev152+g278252a.d20200713


In [2]:
# Import libraries
import xarray 
import numpy as np
import pandas as pd
import os,sys
import matplotlib.pyplot as plt
import iris.plot as iplt
import iris.quickplot as qplt
import datetime
import zipfile,shutil
from six.moves import urllib
from glob import glob
%matplotlib inline

In [3]:
# Import tobac itself
import tobac

In [4]:
# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

**Download example data:**

Actual download has to be performed only once for all example notebooks!

In [5]:
data_out='../'

In [6]:
# # Download the data: This only has to be done once for all tobac examples and can take a while
# file_path='https://zenodo.org/record/3195910/files/climate-processes/tobac_example_data-v1.0.1.zip'
# #file_path='http://zenodo..'
# tempfile='temp.zip'
# print('start downloading data')
# request=urllib.request.urlretrieve(file_path,tempfile)
# print('start extracting data')
# shutil.unpack_archive(tempfile,data_out)
# #zf = zipfile.ZipFile(tempfile)
# #zf.extractall(data_out)
# os.remove(tempfile)
# print('data extracted')


**Load Data from downloaded file:**

In [7]:
data_file = os.path.join(data_out,'*','data','Example_input_Precip.nc')
data_file = glob(data_file)[0]

In [8]:
Precip=xarray.open_dataset(data_file)['surface_precipitation_average']

In [9]:
#display information about the iris cube containing the surface precipitation data:
display(Precip)

<xarray.DataArray 'surface_precipitation_average' (time: 47, south_north: 198, west_east: 198)>
[1842588 values with dtype=float32]
Coordinates:
  * time         (time) datetime64[ns] 2013-06-19T20:05:00 ... 2013-06-19T23:55:00
  * south_north  (south_north) int64 281 282 283 284 285 ... 474 475 476 477 478
  * west_east    (west_east) int64 281 282 283 284 285 ... 474 475 476 477 478
    latitude     (south_north, west_east) float32 ...
    longitude    (south_north, west_east) float32 ...
    x            (west_east) float64 ...
    y            (south_north) float64 ...
    x_0          (west_east) int64 ...
    y_0          (south_north) int64 ...
Attributes:
    long_name:  surface_precipitation_average
    units:      mm h-1

In [10]:
#Set up directory to save output and plots:
savedir='Save'
if not os.path.exists(savedir):
    os.makedirs(savedir)
plot_dir="Plot"
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)

**Feature detection:**

Feature detection is perfomed based on surface precipitation field and a range of thresholds

In [11]:
# Dictionary containing keyword options (could also be directly given to the function)
parameters_features={}
parameters_features['position_threshold']='weighted_diff'
parameters_features['sigma_threshold']=0.5
parameters_features['min_num']=3
parameters_features['min_distance']=0
parameters_features['sigma_threshold']=1
parameters_features['threshold']=[1,2,3,4,5,10,15] #mm/h
parameters_features['n_erosion_threshold']=0
parameters_features['n_min_threshold']=3

In [12]:
# get temporal and spation resolution of the data
dxy,dt=tobac.utils.get_spacings(Precip)

In [13]:
# Feature detection based on based on surface precipitation field and a range of thresholds
print('starting feature detection based on multiple thresholds')
Features=tobac.themes.tobac_v1.feature_detection_multithreshold(Precip,dxy,**parameters_features)
print('feature detection done')
Features.to_netcdf(os.path.join(savedir,'Features.nc'))
print('features saved')

starting feature detection based on multiple thresholds
      frame  idx      hdim_1      hdim_2  num  threshold_value  feature  \
0         0    1   50.065727  139.857477    9                1        1   
1         0   15  120.527119  172.500325    4                1        2   
2         0   18  126.779273  145.368401   15                1        3   
3         0   34  111.611369  155.452030    4                2        4   
4         0   35  111.765231  164.938866    8                2        5   
...     ...  ...         ...         ...  ...              ...      ...   
1124     46   90  163.334480   35.049366   47               15     1125   
1125     46   91  177.119093    1.938903   50               15     1126   
1126     46   92  183.889436  100.054108   36               15     1127   
1127     46   93  196.870749  177.405664   10               15     1128   
1128     46   94  196.936190  191.649040    4               15     1129   

                     time              time

**Segmentation:**

Segmentation is performed based on a watershedding and a threshold value:

In [14]:
# Dictionary containing keyword arguments for segmentation step:
parameters_segmentation={}
parameters_segmentation['method']='watershed'
parameters_segmentation['threshold']=1  # mm/h mixing ratio

In [15]:
# Perform Segmentation and save resulting mask to NetCDF file:
print('Starting segmentation based on surface precipitation')
Mask,Features_Precip=tobac.themes.tobac_v1.segmentation(Features,Precip,dxy,**parameters_segmentation)
print('segmentation based on surface precipitation performed, start saving results to files')
Mask.to_netcdf(os.path.join(savedir,'Mask_Segmentation_precip.nc'))                
Features_Precip.to_netcdf(os.path.join(savedir,'Features_Precip.nc'))
print('segmentation surface precipitation performed and saved')


Starting segmentation based on surface precipitation
<xarray.DataArray 'surface_precipitation_average' (time: 47, south_north: 198, west_east: 198)>
array([[[0.000000e+00, 0.000000e+00, ..., 0.000000e+00, 0.000000e+00],
        [0.000000e+00, 0.000000e+00, ..., 0.000000e+00, 0.000000e+00],
        ...,
        [0.000000e+00, 0.000000e+00, ..., 0.000000e+00, 0.000000e+00],
        [0.000000e+00, 0.000000e+00, ..., 0.000000e+00, 0.000000e+00]],

       [[0.000000e+00, 0.000000e+00, ..., 0.000000e+00, 0.000000e+00],
        [0.000000e+00, 0.000000e+00, ..., 0.000000e+00, 0.000000e+00],
        ...,
        [0.000000e+00, 0.000000e+00, ..., 0.000000e+00, 0.000000e+00],
        [0.000000e+00, 0.000000e+00, ..., 0.000000e+00, 0.000000e+00]],

       ...,

       [[1.162095e-03, 1.237040e-03, ..., 0.000000e+00, 0.000000e+00],
        [1.077846e-03, 1.167705e-03, ..., 0.000000e+00, 0.000000e+00],
        ...,
        [0.000000e+00, 0.000000e+00, ..., 1.475261e-01, 7.602954e-02],
        [0.000

**Trajectory linking:**

Trajectory linking is performed using the trackpy library (http://soft-matter.github.io/trackpy). This takes the feature positions determined in the feature detection step into account but does not include information on the shape of the identified objects.

In [16]:
# Dictionary containing keyword arguments for the linking step:
parameters_linking={}
parameters_linking['method_linking']='predict'
parameters_linking['adaptive_stop']=0.2
parameters_linking['adaptive_step']=0.95
parameters_linking['extrapolate']=0
parameters_linking['order']=1
parameters_linking['subnetwork_size']=100
parameters_linking['memory']=0
parameters_linking['time_cell_min']=5*60
parameters_linking['method_linking']='predict'
parameters_linking['v_max']=10
parameters_linking['d_min']=2000


In [17]:
# Perform trajectory linking using trackpy and save the resulting DataFrame:
Track=tobac.themes.tobac_v1.linking_trackpy(Features,Precip,dt=dt,dxy=dxy,**parameters_linking)
Track.to_netcdf(os.path.join(savedir,'Track.nc'))


Frame 46: 18 trajectories present.


**Visualistation:**

    

In [18]:
# Set extent for maps plotted in the following cells ( in the form [lon_min,lon_max,lat_min,lat_max])
axis_extent=[-95,-93.8,29.5,30.6]        

In [ ]:
# Plot map with all individual tracks:
import cartopy.crs as ccrs
fig_map,ax_map=plt.subplots(figsize=(10,10),subplot_kw={'projection': ccrs.PlateCarree()})
ax_map=tobac.plot.map_tracks(Track,axis_extent=axis_extent,axes=ax_map)

In [ ]:
# Create animation showing tracked cells with outline of precipitation features and the and surface precipitation as a background field:
animation_tobac=tobac.plot.animation_mask_field(track=Track,features=Features,field=Precip,mask=Mask,
                                          axis_extent=axis_extent,#figsize=figsize,orientation_colorbar='horizontal',pad_colorbar=0.2,
                                          vmin=0,vmax=60,extend='both',cmap='Blues',
                                          interval=500,figsize=(10,10),
                                          plot_outline=True,plot_marker=True,marker_track='x',plot_number=True,plot_features=True)

In [ ]:
# Display animation:
from IPython.display import HTML, Image, display
HTML(animation_tobac.to_html5_video())

In [ ]:
# # Save animation to file
# savefile_animation=os.path.join(plot_dir,'Animation.mp4')
# animation_tobac.save(savefile_animation,dpi=200)
# print(f'animation saved to {savefile_animation}')

In [ ]:
# Lifetimes of tracked features:
fig_lifetime,ax_lifetime=plt.subplots()
tobac.plot.plot_lifetime_histogram_bar(Track,axes=ax_lifetime,bin_edges=np.arange(0,120,10),density=False,width_bar=8)
ax_lifetime.set_xlabel('lifetime (min)')
ax_lifetime.set_ylabel('counts')
         